# Imports

In [1]:
import torch as t 
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from datasets import load_dataset
from datasets import Dataset

from tqdm import tqdm

from sloth_parse import parse_bytearray_string_to_sloth_mlp, parse_bytearray_string_to_sloth_cnn

# Data Preprocessing Functions
Can ignore all of this if just training

In [2]:
def process_datapoint(datapoint: dict[str, any]) -> dict[str, any]:
    """
    Map function for the dataset to convert the data into a format that can be used by the model.
    (viz. convert the state from a bytearray string to a tensor)
    Args:
        datapoint: the datapoint to be processed

    Returns:

    """
    game_state_tensor = parse_bytearray_string_to_sloth_cnn(datapoint['state'])
    
    wins = int(datapoint['num_wins'])
    losses = int(datapoint['num_losses'])
    draws = int(datapoint['num_draws'])
    num_games = wins + losses + draws
    score = t.tensor(0.0 if num_games == 0 else (wins - losses) / num_games)
    # this score function attempts to normalize the score to be between -1 and 1, like an evaluation function should
    
    return {
        'tensor_state': game_state_tensor,
        'score': score
    }
    
    

In [3]:
def get_dataset(split: str) -> Dataset:
    if split not in ['train', 'validation', 'test']:
        raise ValueError(f"Invalid split: {split}")
    
    dataset = load_dataset('markstanl/u3t', split=split)
    return dataset

In [4]:
train = get_dataset('train')
test = get_dataset('test')
val = get_dataset('validation')

In [5]:
def get_tensor_dataset(dataset: Dataset) -> Dataset:
    dataset = dataset.remove_columns(['num_visits', 'actions', 'depth'])
    
    dataset = dataset.map(process_datapoint)
    dataset.set_format(type='torch', columns=['tensor_state'])
        
    dataset = dataset.remove_columns(['state', 'num_wins', 'num_losses', 'num_draws'])
    
    return dataset
    
def save_tensor_dataset(dataset: Dataset, split: str):
    dataset.to_parquet(f'sloth_{split}.parquet')
    
def make_tensor_dataset(dataset: Dataset, split: str):
    tensor_dataset = get_tensor_dataset(dataset)
    save_tensor_dataset(tensor_dataset, split)

In [7]:
make_tensor_dataset(train, 'train')
make_tensor_dataset(test, 'test')
make_tensor_dataset(val, 'validation')

Map:   0%|          | 0/5601458 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/5602 [00:00<?, ?ba/s]

Map:   0%|          | 0/1600367 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1601 [00:00<?, ?ba/s]

Map:   0%|          | 0/800165 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/801 [00:00<?, ?ba/s]

Though, run this block to get the data loader from your local dataset

In [5]:
def get_local_data_loader(split: str, cnn: bool=False) -> t.utils.data.DataLoader:
    """
    Get a DataLoader for the given split of the data. Converts the data to tensors.
    Args:
        split: the split of the data to get the DataLoader for
        cnn: boolean to use the cnn split

    Returns:
        DataLoader: the DataLoader for the given split
    """
    if cnn:
        dataset = Dataset.from_parquet(f'sloth_data/sloth_{split}_cnn.parquet')
    else:
        dataset = Dataset.from_parquet(f'sloth_data/sloth_{split}.parquet')
    dataset.set_format(type='torch', columns=['tensor_state', 'score'])
    
    loader = t.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)
    return loader

def get_loader_from_dataset(dataset: Dataset) -> t.utils.data.DataLoader:
    dataset.set_format(type='torch', columns=['tensor_state', 'score'])
    
    loader = t.utils.data.DataLoader(dataset, batch_size=256, shuffle=True)
    return loader

def get_local_dataset(split: str) -> Dataset:
    return Dataset.from_parquet(f'sloth_data/sloth_{split}_cnn.parquet')

# Understanding Data Conversion
I am having trouble with the tensors in the dataset. I realized the issue was with loading datasets as tensors from the parquet file, the solution is to set the format of the dataset to torch when loading.

In [131]:
train_test = train[:10]
train_test_dataset = Dataset.from_dict(train_test)

tensor_dataset = get_tensor_dataset(train_test_dataset)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Column 'state': string
Column 'num_wins': int64
Column 'num_draws': int64
Column 'num_losses': int64
Column 'tensor_state': list
Column 'score': float32


In [136]:
make_tensor_dataset(train_test_dataset, 'train_test')
loader = get_local_data_loader('train_test')

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [137]:
batch = next(iter(loader))

In [138]:
example = batch['tensor_state'][0]
print(type(example))

<class 'torch.Tensor'>


In [140]:
print(example.shape)

torch.Size([4, 81])


In [149]:
tensors = batch['tensor_state']
print(model(tensors))

tensor([[[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[1., 0., 0.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 1., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [1., 1., 1.,  ..., 1., 1., 1.]],

        [[0., 0., 0.,  ..., 0., 0., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0

# Evaluation

In [ ]:
test_loader = get_local_data_loader('test')

In [ ]:
evaluate(model, test_loader)

# Visualization

In [3]:
test_set = get_local_dataset('test')


Dataset({
    features: ['tensor_state', 'score'],
    num_rows: 1600367
})


In [4]:
scores = test_set['score']

In [9]:
import matplotlib.pyplot as plt
import numpy as np

In [4]:
print(scores)
counts, bins = np.histogram(scores)
plt.stairs(counts, bins)
plt.show()


NameError: name 'scores' is not defined

In [8]:
train_set_test = get_local_dataset('train')

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [16]:
hf_set = load_dataset('markstanl/u3t', data_dir='data/state_eval', split='train')

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Loading dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [10]:
print(train_set_test)

Dataset({
    features: ['tensor_state', 'score'],
    num_rows: 5601458
})


In [14]:
print(train_set_test)
print(train_set_test.features.items())

Dataset({
    features: ['tensor_state', 'score'],
    num_rows: 5601458
})
dict_items([('tensor_state', Sequence(feature=Sequence(feature=Sequence(feature=Value(dtype='int32', id=None), length=-1, id=None), length=-1, id=None), length=-1, id=None)), ('score', Value(dtype='float32', id=None))])
